In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = r'C:\Users\TH\Desktop\NN\cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def load_data(filename):#从文件里读取数据
    with open(filename,'rb') as f:
        data =  pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            
        self._data = np.vstack(all_data)
        self._data = self._data 
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data
            
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)#混排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):#返回batch_size个样本
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

    
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

#batch_data,batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)  #测试CifarData类可以正常工作


E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']
(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [2]:
batch_size = 20
x = tf.placeholder(tf.float32,[batch_size,3072])
y = tf.placeholder(tf.int64,[batch_size])    


x_image = tf.reshape(x,[-1,3,32,32])
x_image = tf.transpose(x_image,perm=[0,2,3,1])#交换通道

x_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
result_x_image_arr = []

for x_single_image in x_image_arr:
    # x_single_image: [1, 32, 32, 3] -> [32, 32, 3]
    x_single_image = tf.reshape(x_single_image, [32, 32, 3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)#不支持batch,所以只能拆分
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)
    data_aug_3 = tf.image.random_contrast(
        data_aug_2, lower=0.2, upper=1.8)
    x_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_x_image_arr.append(x_single_image)
result_x_images = tf.concat(result_x_image_arr, axis=0)

normal_result_x_images = result_x_images / 127.5 - 1


conv1_1 = tf.layers.conv2d(normal_result_x_images,32,(3,3),padding='same',activation=tf.nn.relu,name='conv1_1')
conv1_2 = tf.layers.conv2d(conv1_1,32,(3,3),padding='same',activation=tf.nn.relu,name='conv1_2')
pooling1 = tf.layers.max_pooling2d(conv1_2,(2,2),(2,2),name = 'pool1') #16*16


conv2_1 = tf.layers.conv2d(pooling1,32,(3,3),padding='same',activation=tf.nn.relu,name='conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,32,(3,3),padding='same',activation=tf.nn.relu,name='conv2_2')
pooling2 = tf.layers.max_pooling2d(conv2_2,(2,2),(2,2),name = 'pool2') #8*8


conv3_1 = tf.layers.conv2d(pooling2,32,(3,3),padding='same',activation=tf.nn.relu,name='conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,32,(3,3),padding='same',activation=tf.nn.relu,name='conv3_2')
pooling3 = tf.layers.max_pooling2d(conv3_2,(2,2),(2,2),name = 'pool3') #4*4


#全连接层
flatten = tf.layers.flatten(pooling3)#把二维图像变成一维向量[None,4*4*32]
y_ = tf.layers.dense(flatten,10)

#交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#这个函数可以计算softmax 计算onehot 计算ylogy_

predict = tf.argmax(y_,1)
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [3]:
#指定变量
def variable_summary(var,name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))#求均值
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min',tf.reduce_min(var))#求最小值
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.histogram('histogram',var)#打印直方图
        
with tf.name_scope('summary'):
    variable_summary(conv1_1,'conv1_1')
    variable_summary(conv1_2,'conv1_2')
    variable_summary(conv2_1,'conv2_1')
    variable_summary(conv2_2,'conv2_2')
    variable_summary(conv3_1,'conv3_1')
    variable_summary(conv3_2,'conv3_2')
    

loss_summary = tf.summary.scalar('loss',loss)#loss和accuracy都是标量所以用loss
accuracy_summary = tf.summary.scalar('accuracy',accuracy)
inputs_summary = tf.summary.image('input_image',result_x_images)#x_image是归一化以后的值

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary,accuracy_summary])

#指定输出文件
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir  = os.path.join(LOG_DIR,run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir,'train')
test_log_dir = os.path.join(run_dir,'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)


In [4]:
init = tf.global_variables_initializer()

batch_size = 20
train_steps = 10000
test_steps = 100
output_summary_every_step = 100

with tf.Session() as sess:
    init.run()
    train_writer = tf.summary.FileWriter(train_log_dir,sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)#无sess.graph则无计算图
    
    fixed_test_batch_data,fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        
        #增加训练summary
        eval_ops = [loss,accuracy,train_op]
        should_output_summary = (i+1)/output_summary_every_step==0
        if should_output_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(eval_ops,feed_dict={x:batch_data,y:batch_labels})
        loss_val,accu_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str,i+1)
            test_summary_str = sess.run([merged_summary_test],feed_dict={x:fixed_test_batch_data,y:fixed_test_batch_labels})[0]
            test_writer.add_summary(test_summary_str,i+1)
        
        if i%500 == 0 :
            print('[Train] step',i,' loss=',loss_val,' acc=',accu_val)
            
        if i%5000==0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test step',i+1,' acc=',test_acc)

[Train] step 0  loss= 2.3009796  acc= 0.05
(10000, 3072)
(10000,)
Test step 1  acc= 0.10200000000000001


KeyboardInterrupt: 